In [1]:
import pandas as pd

In [6]:
data = pd.read_csv("cleaned_combined_lendingclub.csv",low_memory=True)

C:\Users\avida\AppData\Local\Temp\ipykernel_13780\1054590859.py:1: DtypeWarning: Columns (3,7,10,11,12,13,15,16,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("cleaned_combined_lendingclub.csv",low_memory=True)


In [8]:
data.shape

(29909442, 24)

In [13]:
data.loc[data['accepted_or_rejected'] == 'rejected', 'loan_status'] = 'rejected'



In [14]:
df = data.dropna(subset=["Amount Requested", "Debt-To-Income Ratio", "accepted_or_rejected"])
print(f"Remaining rows: {len(df)}")


Remaining rows: 29907698


In [15]:
df.shape

(29907698, 24)

In [16]:
df.to_csv("lendingclub_filtered.csv", index=False)


In [17]:
import os
os.system("java -version")


0

In [18]:
import pandas as pd
import numpy as np

# Load just the header first to know what you're working with
header = pd.read_csv("lendingclub_filtered.csv", nrows=0).columns.tolist()

# Load a random sample of 50,000 rows
df_sample_50k = pd.read_csv("lendingclub_filtered.csv", usecols=header, skiprows=lambda i: i > 0 and np.random.rand() > 50000 / 2500000)

# Save to CSV
df_sample_50k.to_csv("sample_lendingclub_50k.csv", index=False)

# To sample 100,000 instead:
# df_sample_100k = pd.read_csv("cleaned_lendingclub_filtered.csv", usecols=header, skiprows=lambda i: i > 0 and np.random.rand() > 100000 / 2500000)
# df_sample_100k.to_csv("sample_lendingclub_100k.csv", index=False)

print("✅ Sample saved to 'sample_lendingclub_50k.csv'")


C:\Users\avida\AppData\Local\Temp\ipykernel_13780\2017621587.py:8: DtypeWarning: Columns (3,7,10,11,12,13,16,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sample_50k = pd.read_csv("lendingclub_filtered.csv", usecols=header, skiprows=lambda i: i > 0 and np.random.rand() > 50000 / 2500000)


✅ Sample saved to 'sample_lendingclub_50k.csv'


In [19]:
import pandas as pd

# Define labels
default_statuses = {"Charged Off", "Default", "Late(16-30 days)", "Late (31-120 days)", "In Grace Period"}
non_default_statuses = {"Fully Paid", "Current"}

# Prepare containers
default_rows = []
non_default_rows = []

# How many rows we want from each class
target_per_class = 25000

# Load in chunks
chunk_size = 100_000
reader = pd.read_csv("lendingclub_filtered.csv", chunksize=chunk_size)

for chunk in reader:
    if "loan_status" not in chunk.columns:
        raise ValueError("loan_status column missing")

    # Filter rows into groups
    default_chunk = chunk[chunk["loan_status"].isin(default_statuses)]
    non_default_chunk = chunk[chunk["loan_status"].isin(non_default_statuses)]

    default_rows.append(default_chunk)
    non_default_rows.append(non_default_chunk)

    # Stop early if we’ve gathered enough
    if sum(len(df) for df in default_rows) > target_per_class and sum(len(df) for df in non_default_rows) > target_per_class:
        break

# Concatenate and sample
df_default = pd.concat(default_rows).sample(n=target_per_class, random_state=42)
df_non_default = pd.concat(non_default_rows).sample(n=target_per_class, random_state=42)

# Combine and shuffle
df_sample_50k = pd.concat([df_default, df_non_default]).sample(frac=1, random_state=42)

# Save
df_sample_50k.to_csv("sample_lendingclub_50k.csv", index=False)
print("✅ Balanced sample saved to 'sample_lendingclub_50k.csv'")


✅ Balanced sample saved to 'sample_lendingclub_50k.csv'
